# Project Part2

In [1]:
import math 
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
import itertools
import time

## 1. 
Write a function decoder(A,C,p,o) that takes as inputs:  
• An n × n state transmission matrix A for a hidden Markov model  
• An m × n emission probability matrix C  
• An initial state probability vector p ∈ R  
• A length t sequence of observations o.  

Your function should run the decoding algorithm and return a length t sequences of states y, that corresponds to the most likely sequence of states to have produced the given sequence of observations for this hidden Markov model.
The logic is very similar to the maximum function that you wrote for part 1 of that project. You can
modify that code for this part

In [16]:
def max(A,C,p,o):
    #create a matrix to track
    track = np.zeros((len(o), np.shape(A)[0]))
    
    #code from maximum function
    gamma_old = C[o[0]]*p
    for t in range(0, len(o)):
        gamma_new = np.zeros_like(gamma_old)
        for i in range(np.shape(A)[0]):
            gamma_new[i] = np.max(gamma_old * A[i] * C[o[t],i])
            #the maximum gamma
            track[t][i] = np.argmax(gamma_new)
        gamma = gamma_new
    
    return gamma, track

def decoder (A,C,p,o):
    
    gamma, track = max(A,C,p,o)
    #create a new array to store state sequence
    state = []
    
    #range from len(o)-1 to -1, step by minus 1
    for t in range(len(o)-1, -1, -1):
        #the last one should be the index of maximum gamma
        if t == len(o)-1:
            state_max = np.argmax(gamma)
        else:
            #create a new empty matrix 
            new = np.zeros(np.shape(A)[0]-1)
            for i in range(np.shape(A)[0]-1):
                new[i] = A[i, state_max] * track[t][i]
            state_max = np.argmax(new)
        
        #append the state sequence
        state.append(state_max)

    return state

In [17]:
#test
A = np.array([[0.8, 0.2],[0.2,0.8]])
C = np.array([[0.5, 0.75],[0.5, 0.25]])
p = np.array([1,0])
o = np.array([0,0,0])
decoder(A,C,p,o)

AttributeError: 'NoneType' object has no attribute 'reverse'

## 2.
Suppose we play a game with a four-sided die where the player wins if the roll is even (0 or 2) and the
dealer wins in the roll is odd (1 or 3). In this part you will use the function you wrote in problem 1 to
detect potential cheating on the part of the dealer. Suppose the following:  

• The has a fair die (all faces have .25 probability), and a weighted die (faces 0,1,2 each have .12
probability and face 3 has a .64 probability)  
• The dealer always starts with the fair die.  
• After each subsequent roll there is a .1probability that the dealer switches to the weighted die.  
• Once using the weighted die, there is a .1 probability that the dealer switches back to the fair die.  
• The dealer is fearful of being caught. If the dealer switches back to the fair die, they use it for the
rest of the sequence. (So there are at most two total switches)  

Given each of the following sequences of rolls, determine the most likely sequence of states to have
produced the sequence. Assuming this is the correct sequence of states, did the dealer cheat? If so, for
which rolls was the dealer using the weighted die?

In [4]:
A = np.array([[0.9, 0.1],[0.1,0.9]])
C = np.array([[0.25, 0.12],[0.25, 0.12],[0.25,0.12],[0.25,0.64]])
p = np.array([1,0])

### (a) 

In [5]:
o = [0, 3, 2, 2, 1, 2, 1, 1, 1, 1, 1, 0, 1, 3, 3, 1, 3, 3, 1, 1, 3, 2, 1, 1, 2, 2, 0, 0, 3, 2]
print("the sequence of states is: ", decoder(A,C,p,o))

the sequence of states is:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


The dealer did not cheat.

### (b)

In [6]:
o = [0, 1, 3, 0, 1, 2, 3, 0, 2, 1, 3, 3, 3, 0, 3, 2, 3, 3, 3, 2, 2, 0, 1, 0, 2, 3, 0, 1, 1, 3]
print("the sequence of states is: ", decoder(A,C,p,o))

the sequence of states is:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## 3. 
We will now use our algorithm for a gene identification problem.
Suppose that have a DNA sequence of four nucleotides (A,C,G,T). Suppose that each sequence consists
of an exon (a sequence of the DNA that encodes a protein), a 5’ receptor site (a single nucleotide) and
an intron (an interrupter sequence that does not encode a protein). Assume the following probabilites:  

• We always start with the exon.  
• Given a nucleotide that is part of the exon, the next nucleotide will be part of the exon with
probability .9, and will be the 5’ receptor with probability .  
• The 5’ receptor is always exactly one nucleotide long. Once we pass it, all subsequent nucleotides
are part of the intron.  

Further assume the following emission probabilities:  
• In the exon, all four nucleotides are equally likely.  
• The 5’ nucleotide is a G with probability .95, and an A with probability .05.  
• In the intron, the nucleotides A and T each have probability .4, and C and G have probability .  

In [7]:
A = np.array([[0.9, 0.0, 0.0], [0.1, 0.0, 0.0], [0.0, 1.0, 1.0]])
C = np.array([[0.25, 0.05, 0.4], [0.25, 0,0.1], [0.25,0.95,0.1], [0.25,0,0.4]])
p = np.array([1,0,0])

Given the DNA sequence:  
CT T CAT GT T AAAGCAGACGT AAGT CA  
identify the initial segment that is most likely to be the exon.  

In [8]:
o = [1,3,3,1,0,3,2,3,3,0,0,0,2,1,0,2,0,1,2,3,0,0,2,3,1,0]
print("the sequence of states is: ", decoder(A,C,p,o))

the sequence of states is:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [9]:
for i in range(8, 0, -1):
    print(i) 
    
for i in range(7):
    print(i)

8
7
6
5
4
3
2
1
0
1
2
3
4
5
6
